In [ ]:
from __future__ import print_function
from __future__ import division
from sklearn.metrics import precision_recall_fscore_support
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
data_dir = "main_set"


num_classes = 4

batch_size = 8

num_epochs = 10

feature_extract = False

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "densenet":
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
def transformations(input_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'validation': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    print("Initializing Datasets and Dataloaders...")


    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test', 'validation']}

    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'test', 'validation']}

    print("Finished initializing Datasets and Dataloaders.")
    
    return dataloaders_dict

# F1 testing




In [ ]:
for model_name in ['alexnet_not_pretrainred', 'alexnet', 'vgg', 'densenet', 'resnet']:
    
    if model_name == 'alexnet_not_pretrainred':
        model_ft, input_size = initialize_model('alexnet', 4, False, use_pretrained=False)
    else:
        model_ft, input_size = initialize_model(model_name, 4, False, use_pretrained=True)
    model_ft.load_state_dict(torch.load('model_{}.pt'.format(model_name)))
    
    dataloaders_dict = transformations(input_size)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    running_loss = 0
    running_corrects = 0
    max_correct = 0

    for inputs, labels in dataloaders_dict['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            model_ft.eval()
            outputs = model_ft(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += precision_recall_fscore_support(labels.data, preds, average='macro', zero_division=0)[2]
        max_correct += 1

    score = running_corrects / max_correct

    print('Model {} F-1 score: {:.4f}'.format(model_name, score))
